In [1]:
import os
import tempfile
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.conf import SparkConf
from pyspark import SparkContext
from minio import Minio
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Inicializar SparkSession
try:
    sc = SparkContext.getOrCreate()
    sc.stop()
except:
    pass
import time
time.sleep(1)
conf = SparkConf().setAppName("LimpiezaSilver").setMaster("local[*]").set("spark.driver.bindAddress", "127.0.0.1").set("spark.driver.host", "127.0.0.1")
try:
    sc = SparkContext(conf=conf)
    spark = SparkSession(sc)
except:
    spark = SparkSession.builder.config(conf=conf).getOrCreate()
print("✅ Spark iniciado")

AttributeError: 'NoneType' object has no attribute 'sc'

In [ ]:
# Configuración MinIO
MINIO_ENDPOINT = os.environ.get("MINIO_ENDPOINT", "localhost:9000")
MINIO_ACCESS_KEY = os.environ.get("MINIO_ACCESS_KEY", "minioadmin")
MINIO_SECRET_KEY = os.environ.get("MINIO_SECRET_KEY", "minioadmin")
MINIO_BUCKET_BRONCE = os.environ.get("MINIO_BUCKET", "meteo-bronze")
MINIO_BUCKET_SILVER = "meteo-silver"

minio_client = Minio(MINIO_ENDPOINT, access_key=MINIO_ACCESS_KEY, secret_key=MINIO_SECRET_KEY, secure=False)
print("✅ MinIO conectado")

# Crear bucket Silver si no existe
try:
    minio_client.make_bucket(MINIO_BUCKET_SILVER)
    print(f'✅ Bucket {MINIO_BUCKET_SILVER} creado')
except:
    print(f'✅ Bucket {MINIO_BUCKET_SILVER} ya existe')

In [ ]:
# Cargar archivo más reciente de Bronze
archivo_reciente = None
try:
    print("📥 Buscando archivos Bronce...")
    objects = minio_client.list_objects(MINIO_BUCKET_BRONCE, recursive=True)
    archivos_csv = [obj.object_name for obj in objects if obj.object_name.endswith(".csv")]
    if archivos_csv:
        archivo_reciente = sorted(archivos_csv)[-1]
        print(f"✅ Cargando: {archivo_reciente}")
        temp_dir = tempfile.gettempdir()
        temp_file = os.path.join(temp_dir, archivo_reciente.split("/")[-1])
        minio_client.fget_object(MINIO_BUCKET_BRONCE, archivo_reciente, temp_file)
        df = spark.read.csv(temp_file, header=True, inferSchema=True)
        print(f"✅ Cargados {df.count()} registros")
    else:
        print("⚠️ Sin archivos en Bronce")
        df = spark.createDataFrame([(1, 25.5, 60)], ["id", "temperature", "humidity"])
except Exception as e:
    print(f"⚠️ Error: {e}")
    df = spark.createDataFrame([(1, 25.5, 60)], ["id", "temperature", "humidity"])

print(f"\n📊 DataFrame: {df.count()} registros")
df.show()

In [ ]:
# Limpieza de datos
print("\n🧹 LIMPIEZA DE DATOS...")

# Eliminar columnas innecesarias
df = df.drop('pressure', 'uv_level', 'pm25', 'rain_raw', 'wind_raw', 'vibration')

# Eliminar duplicados
df = df.dropDuplicates()

# Descomponer timestamp en columnas separadas
from pyspark.sql.functions import year, month, dayofmonth, hour, minute, second

# Buscar columnas de tipo timestamp
timestamp_cols = [field.name for field in df.schema.fields if "timestamp" in field.name.lower()]

for ts_col in timestamp_cols:
    df = df.withColumn(f"{ts_col}_anio", year(col(ts_col))) \
            .withColumn(f"{ts_col}_mes", month(col(ts_col))) \
            .withColumn(f"{ts_col}_dia", dayofmonth(col(ts_col))) \
            .withColumn(f"{ts_col}_hora", hour(col(ts_col))) \
            .withColumn(f"{ts_col}_minuto", minute(col(ts_col))) \
            .withColumn(f"{ts_col}_segundo", second(col(ts_col)))

print(f"\n✅ {df.count()} registros limpios")
df.show(3)

In [ ]:
# Guardar en Silver (2 archivos: nombre tabla + nombre estándar para Power BI)

print("\n💾 GUARDANDO EN SILVER...")

tabla = archivo_reciente.split('_bronce_')[0] if archivo_reciente and '_bronce_' in archivo_reciente else 'datos'
archivo_silver = f'{tabla}_silver.csv'
archivo_standard = 'datos_principales_silver.csv'  # Nombre estándar para Power BI

try:
    import io
    from pyspark.sql.types import TimestampType
    from pyspark.sql.functions import col

    # --- SOLUCIÓN DEL ERROR ---
    # Convertimos las columnas de fecha (Timestamp) a String DENTRO de Spark.
    # Esto evita que Pandas intente hacer la conversión estricta de datetime64[ns] que está fallando.
    df_export = df  # Creamos una copia para exportar
    for field in df.schema.fields:
        if isinstance(field.dataType, TimestampType):
            # Forzamos el formato string para el CSV
            df_export = df_export.withColumn(field.name, col(field.name).cast("string"))
    # --------------------------

    # Convertir a pandas y generar CSV en memoria (evita Hadoop)
    # Ahora usamos df_export en lugar de df
    pdf = df_export.toPandas()
    
    csv_buffer = pdf.to_csv(index=False)
    csv_bytes = io.BytesIO(csv_buffer.encode('utf-8'))
    
    # Subir con nombre de tabla específico
    csv_bytes_1 = io.BytesIO(csv_buffer.encode('utf-8'))
    minio_client.put_object(
        MINIO_BUCKET_SILVER, 
        archivo_silver, 
        csv_bytes_1,
        length=len(csv_buffer.encode('utf-8')),
        content_type="text/csv"
    )
    print(f"✅ {archivo_silver} actualizado en Silver")
    
    # Subir con nombre estándar para Power BI (meteo-silver/datos_principales_silver.csv)
    csv_bytes_2 = io.BytesIO(csv_buffer.encode('utf-8'))
    minio_client.put_object(
        MINIO_BUCKET_SILVER, 
        archivo_standard, 
        csv_bytes_2,
        length=len(csv_buffer.encode('utf-8')),
        content_type="text/csv"
    )
    print(f"✅ {archivo_standard} actualizado en Silver")
    
    print("\n" + "="*70)
    print("✅ LIMPIEZA DE SILVER COMPLETADA")
    print("="*70)
    print(f"📍 Origen: {archivo_reciente}")
    print(f"📍 Destino (tabla): meteo-silver/{archivo_silver}")
    print(f"📍 Destino (Power BI): meteo-silver/{archivo_standard}")
    print(f"📊 Registros: {df.count()} limpios y sin duplicados")
    print("="*70)
        
except Exception as e:
    error_msg = str(e)
    if "UnsatisfiedLinkError" in error_msg or "NativeIO" in error_msg:
        print(f"⚠️  Warning Hadoop ignorado (no crítico)")
        print("✅ LIMPIEZA DE SILVER COMPLETADA")
    else:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()